Free[S,A] 

* Free - This is the program
* S - This is the language
* A - This is the type of a vlaue it will produce once you run the programm.


## 15 - Free Monads

Base on [this blog post](https://serokell.io/blog/introduction-to-free-monads).

Remember that for a set $A$, a free monoid is a list of $A$. Such free monoid
can be coded as the following functor:
```haskell
List a = Nil | Cons a (List a)
```

Similarly, if we have
an endofunctor $F:\mathbf{Set} \to \mathbf{Set}$, then we can think of a free
monad over $F$ as a "list" of applications of $F$.
For example, consider a functor 
```haskell
F a := Nil | Cons a
```
For a value `x :: Int`, we have that  `F(x)` is either `Nil` or `Cons Int`.
A free monad over such functor would the functor
```haskell
(Free F) a := Pure a | Free (F (Free F a))
```

Suppose we have a functor:
```haskell
data F a = One a | Two a a | Two' a a | Three Int a a a
```

We want to construct tree whose leaves have type `a` and whose
nodes are one of the "subtypes" `One`, `Two`, `Two'` or `Three`, e.g.
```
     Two
    /   \
 One   Three
  |   / / | \
  a  2 a  a Two'
            / \
           a   a
```

Let's start by creating the `F` functor. 

In [1]:
using MLStyle
@data F{a} begin
    One(::a)
    Two(::a,::a)
    Two_(::a,::a)
    Three(Int, ::a, ::a,::a)
end
fmap(f::Function, x::One) = One(f(x._1))
fmap(f::Function, x::Two) = Two(f(x._1),f(x._2))
fmap(f::Function, x::Two_) = Two_(f(x._1),f(x._2))
fmap(f::Function, x::Three) = Three(x._1,f(x._2),f(x._3), f(x._4));

In [2]:
fmap(x->x+1,One(10))
fmap(x->x+1,Two(10,0))
fmap(x->x+1,Two_(10,0))
fmap(x->x+1,Three(3,0.,0.,0.))

Three{Float64}(3, 1.0, 1.0, 1.0)

Now,we want to create the tree example above. 

In [3]:
Two(One("f"), Three(2,"a","b",Two_("c","d")))

LoadError: MethodError: no method matching Three(::Int64, ::String, ::String, ::Two_{String})

[0mClosest candidates are:
[0m  Three(::Int64, ::a, ::a, [91m::a[39m) where a
[0m[90m   @[39m [35mMain[39m [90m[4mIn[1]:6[24m[39m


The code above does not work. Note that
`Three` is taking an `Int`, two arguments of type `String` and a last argument
of type `Two_`. 
This goes agains the definition of `Three(_1::Int,_2::a,_3::a,_4::a)`. 

The solution to this is creating a new functor `FreeF`, where
each value `a` is wrapped into a container `Pure`, and each
of the "subfunctors" (i.e. `FreeOne`, `FreeTwo`, `FreeTwo_`, `FreeThree`)
can receive a value either `Pure{a}` or `Free{a}`.

In the example below, we will construct the `FreeF` functor manually. Yet,
the idea is that `Free` should be a functor itself, such that `Free ∘ F`
would define `FreeF`.

The `FreeF` is a functor with an `fmap` that inherits the `fmap` from `F`.

In [64]:
abstract type FreeF end

struct Pure{a} <: FreeF
    _1::a
end
struct Roll{a} <: FreeF
    _1::F{a}
end

# FreeF{a} = Union{Pure{a},Roll{F,a}}

Pure(One(1))
x = Roll(One(Pure(1)))

fmap(f::Function, x::Pure) = Pure(f(x._1))
fmap(f::Function, x::Roll) = Roll(fmap(y->fmap(f,y),x._1))

μ(x::Pure) = x._1
μ(x::Roll) = Roll(fmap(μ, x._1))

liftFreeF(x::F) = Roll(fmap(x->Pure(x),x))
foldFreeF(alg::Function, x::Pure) = x._1
foldFreeF(alg::Function, x::FreeF) = alg(fmap(y -> foldFreeF(alg, y), x._1))

evalsum(x::One{<:Real}) = x._1
evalsum(x::Two{<:Real}) = x._1 + x._2
evalsum(x::Two_{<:Real}) = x._1 - x._2
evalsum(x::Three{<:Real}) = x._1 + x._2 + x._3 + x._4


evaldraw(x::One{<:Real}) = string(x._1)
evaldraw(x::Two{<:Real}) = string(x._1) * string(x._2)
evaldraw(x::Two_{<:Real}) = string(x._1) * string(x._2)
evaldraw(x::Three{<:Real}) = string(x._1) * string(x._2) * string(x._3) * string(x._4)

evaldraw (generic function with 4 methods)

In [65]:
foldFreeF(evaldraw,
    μ(liftFreeF(
            Two(
                liftFreeF(Two(1,2)),
                liftFreeF(One(1)
                    )
                )
        )
    )
)

LoadError: MethodError: no method matching evaldraw(::Two{String})

[0mClosest candidates are:
[0m  evaldraw([91m::One{<:Real}[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[64]:31[24m[39m
[0m  evaldraw([91m::Two{<:Real}[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[64]:32[24m[39m
[0m  evaldraw([91m::Two_{<:Real}[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[64]:33[24m[39m
[0m  ...


In [43]:
foldFreeF(evalsum,liftFreeF(Two(1,2)))

3

In [ ]:
result = Pure(One(RollF(One(Pure(1)))))

μ(result)

In [ ]:
# Tree = FreeTwo(FreeOne(Pure("f")),FreeThree(2,Pure("a"),Pure("b"),FreeTwo_(Pure("c"),Pure("d"))));